In [1]:
import geopandas as gpd
import yaml, re, unicodedata
from pathlib import Path
import pandas as pd
import requests

In [2]:
shapefile = "/home/aninotna/magister/tesis/justh2_pipeline/data/generadoras/Solares.shp"
OUT = "sites_solares.csv"
NAME_FIELD = "NOMBRE"
gdf = gpd.read_file(shapefile)
print("📌 Columnas disponibles en el SHP:")
print(gdf.columns.tolist())
print("\n📌 Primeras filas para ver los datos:")
print(gdf.head())

📌 Columnas disponibles en el SHP:
['ID_GE_SOLA', 'NOMBRE', 'PROPIEDAD', 'TIPO', 'MEDIO_GENE', 'COMBUSTIBL', 'POTENCIAMW', 'UNIDADES', 'RCA', 'SIST_ELECT', 'ESTADO', 'F_OPERACIO', 'COORD_ESTE', 'COORD_NORT', 'HUSO', 'DATUM', 'REGION', 'PROVINCIA', 'COMUNA', 'FUENTE_BAS', 'FECH_CREA', 'FECH_ACT', 'geometry']

📌 Primeras filas para ver los datos:
   ID_GE_SOLA              NOMBRE            PROPIEDAD          TIPO  \
0         410  LOMA LOS COLORADOS     KDM ENERGIA S.A.  FOTOVOLTAICO   
1         377      SANTIAGO SOLAR  SANTIAGO SOLAR S.A.  FOTOVOLTAICO   
2         836      PALTO SUNLIGHT   PALTO SUNLIGHT SPA  FOTOVOLTAICO   
3         167   FUSTER DEL VERANO    LASCAR ENERGY SPA  FOTOVOLTAICO   
4          70              COLINA  CHACABUCO SOLAR SPA  FOTOVOLTAICO   

  MEDIO_GENE COMBUSTIBL  POTENCIAMW  UNIDADES               RCA SIST_ELECT  \
0        PMG  NO APLICA       0.764         0         NO APLICA        SEN   
1       None  NO APLICA      90.830         0          310/2016  

In [3]:
locations = {}

for idx, row in gdf.iterrows():
    nombre = "PV_" + str(row["NOMBRE"]).strip().replace(" ", "_").replace("/", "_")
    cap = float(row["POTENCIAMW"]) if row["POTENCIAMW"] else 0.0
    lon, lat = row.geometry.x, row.geometry.y

    locations[nombre] = {
        "coordinates": [float(lon), float(lat)],
        "techs": {
            "solar_pv": {
                "constraints": {
                    "energy_cap": cap
                }
            }
        }
    }

yaml_data = {"locations": locations}

with open("locations_solares.yml", "w", encoding="utf-8") as f:
    yaml.dump(yaml_data, f, sort_keys=False, allow_unicode=True)

print("✅ Archivo YAML generado: locations_solares.yml")

✅ Archivo YAML generado: locations_solares.yml


In [4]:

# Asegurar WGS84 (lat/lon)
if gdf.crs and gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)


In [ ]:
rows = []
for _, r in gdf.iterrows():
    geom = r.geometry
    if geom is None or geom.is_empty or geom.geom_type != "Point":
        continue
    lon, lat = geom.x, geom.y
    name = str(r.get(NAME_FIELD, "")).strip()
    if not name:
        continue
    rows.append((lat, lon, name))

# Quitar duplicados (por nombre y coords)
df = pd.DataFrame(rows, columns=["lat", "lon", "name"]).drop_duplicates()

# Guardar SIN encabezado, como en tu ejemplo
Path(OUT).write_text(
    "\n".join([f"{lat:.6f}, {lon:.6f}, {name}" for lat, lon, name in df.values]),
    encoding="utf-8"
)

print(f"✅ CSV creado: {OUT} (formato: lat, lon, nombre; sin header)")

✅ CSV creado: sites_solares.csv (formato: lat, lon, nombre; sin header)


In [23]:
import yaml
from pathlib import Path

YML = "locations_solares.yml"  # generado por tu script
OUT = "sites_solares_from_yaml.csv"

data = yaml.safe_load(Path(YML).read_text(encoding="utf-8"))
locs = data.get("locations", {})

lines = []
for name, payload in locs.items():
    coords = payload.get("coordinates", None)
    if not coords or len(coords) < 2:
        continue
    lon, lat = float(coords[0]), float(coords[1])  # tu YAML guarda [lon, lat]
    # Si quieres nombres “bonitos”:
    pretty = name.strip()
    lines.append(f"{lat:.6f}, {lon:.6f}, {pretty}")

Path(OUT).write_text("\n".join(lines), encoding="utf-8")
print(f"✅ CSV creado: {OUT} (formato: lat, lon, nombre; sin header)")

✅ CSV creado: sites_solares_from_yaml.csv (formato: lat, lon, nombre; sin header)


In [9]:
OUT_DIR = Path("pv_profiles")
OUT_DIR.mkdir(exist_ok=True)
PV_CAP_KW = 1000  # Potencia pico usada para cálculo (kWp)

In [ ]:
def get_pvgis_profile(lat, lon, name):
    url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
    params = {
        "lat": lat,
        "lon": lon,
        "raddatabase": "PVGIS-ERA5",
        "browser": "0",
        "outputformat": "jzson",
        "startyear": 2020,
        "endyear": 2020,
        "pvtechchoice": "crystSi",
        "peakpower": PV_CAP_KW,
        "pvcalculation": 1,          # => fuerza cálculo FV para que venga 'P'
        "loss": 14,                  # % pérdidas sistema
        "angle": 20,                 # inclinación del panel
        "aspect": 180,               # => hemisferio sur: orientar al norte (PVGIS: 0=sur, 180=norte)
        "optimalangles": 0,
        "hourlyoptimalangles": 0,
        "usehorizon": 1
    }
    r = requests.get(url, params=params, timeout=60)  # => timeout por robustez
    r.raise_for_status()
    data = r.json()
    hourly_data = data["outputs"]["hourly"]
    df = pd.DataFrame(hourly_data)

    df["time"] = pd.to_datetime(df["time"], format="%Y%m%d:%H%M", utc=True)  # => parseo explícito

    cols = set(df.columns)
    if "P" in cols:
        df["resource"] = df["P"] / PV_CAP_KW         # => CF 0–1 desde potencia (kW/kWp)
    else:
        pr = 0.83                                     # => performance ratio típico
        if "G(i)" in cols:
            G_POA = df["G(i)"]                       # W/m² en plano del generador
        elif {"Gb(i)", "Gd(i)", "Gr(i)"} <= cols:
            G_POA = df["Gb(i)"] + df["Gd(i)"] + df["Gr(i)"]
        else:
            raise ValueError(f"No viene 'P' ni G(i)/Gb/Gd/Gr. Columnas: {sorted(cols)}")
        df["resource"] = (G_POA * pr) / 1000.0       # => kW/kWp ~ CF
        df.loc[df["resource"] > 1, "resource"] = 1   # => recorte superior

    file_path = OUT_DIR / f"PV_{name}.csv"
    df[["time", "resource"]].to_csv(file_path, index=False)  # => guardar *después* de calcular resource
    print(f"✅ Guardado {file_path}")


In [21]:
# === LOOP SOBRE PLANTAS ===
for _, row in gdf.iterrows():
    name = str(row["NOMBRE"]).strip().replace(" ", "_")
    lon, lat = row.geometry.x, row.geometry.y
    try:
        get_pvgis_profile(lat, lon, name)
    except Exception as e:
        print(f"⚠️ Error con {name}: {e}")


✅ Guardado pv_profiles/PV_LOMA_LOS_COLORADOS.csv
✅ Guardado pv_profiles/PV_SANTIAGO_SOLAR.csv
✅ Guardado pv_profiles/PV_PALTO_SUNLIGHT.csv
✅ Guardado pv_profiles/PV_FUSTER_DEL_VERANO.csv
✅ Guardado pv_profiles/PV_COLINA.csv
✅ Guardado pv_profiles/PV_VILLA_MOSCOSO.csv
✅ Guardado pv_profiles/PV_PULLALLI.csv
✅ Guardado pv_profiles/PV_TRANQUE_DEL_SOL.csv
✅ Guardado pv_profiles/PV_OCOA.csv
✅ Guardado pv_profiles/PV_PEDREROS_SOLAR.csv
✅ Guardado pv_profiles/PV_RODEO.csv
✅ Guardado pv_profiles/PV_LA_LIGUA.csv
✅ Guardado pv_profiles/PV_EL_BOCO.csv
✅ Guardado pv_profiles/PV_CATAN_SOLAR.csv
✅ Guardado pv_profiles/PV_ENCON_SOLAR.csv
✅ Guardado pv_profiles/PV_ANUNUCA_DEL_VERANO.csv
✅ Guardado pv_profiles/PV_SANTA_INES_SOLAR.csv
✅ Guardado pv_profiles/PV_MESETA_DE_LOS_ANDES.csv
✅ Guardado pv_profiles/PV_PARQUE_SOLAR_OVEJERIA.csv
✅ Guardado pv_profiles/PV_DONA_CARMEN_SOLAR.csv
✅ Guardado pv_profiles/PV_PV_UTFSM_VALPARAISO_VALDES.csv
✅ Guardado pv_profiles/PV_LIQUIDAMBAR.csv
✅ Guardado pv_profiles/PV

In [3]:
import shutil

YML_IN  = Path("locations_solares.yml")
YML_OUT = Path("locations_solares_with_resource.yml")
CSV_DIR = Path("pv_profiles_cf")

In [6]:
def _norm(s: str) -> str:
    # => normaliza: mayúsculas, sin acentos/ñ, underscores
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.upper().strip()
    s = re.sub(r"[^\w]+", "_", s)  # => espacios, guiones, etc. -> "_"
    return s


In [7]:
def _find_or_rename_csv(expected_stem: str) -> str:
    # => intenta encontrar CSV cuyo "stem" normalizado coincida; si existe exacto, lo usa
    expected = CSV_DIR / f"{expected_stem}.csv"
    if expected.exists():
        return str(expected)

    # => buscar candidatos por normalización
    target_norm = _norm(expected_stem)
    candidates = list(CSV_DIR.glob("*.csv"))
    for c in candidates:
        if _norm(c.stem) == target_norm:
            # => renombra al nombre esperado para consistencia
            shutil.move(str(c), str(expected))
            return str(expected)

    # => si no encontró, devuelve ruta esperada (por si luego generas ese CSV)
    return str(expected)


In [8]:
data = yaml.safe_load(YML_IN.read_text(encoding="utf-8"))
locations = data.get("locations", {})

In [9]:
for loc_name, payload in locations.items():
    techs = payload.get("techs", {}) or {}

    # => detecta la tech FV: prioriza 'solar_pv', si no existe, usa 'pv' si está
    tech_key = "solar_pv" if "solar_pv" in techs else ("pv" if "pv" in techs else None)
    if not tech_key:
        continue  # => sin tech FV, salta

    # => nombre de archivo esperado: igual al nombre del location
    expected_stem = loc_name  # p.ej. PV_ALICAHUE
    csv_path = _find_or_rename_csv(expected_stem)

    # => crea nodos intermedios si faltan y setea el resource file
    techs.setdefault(tech_key, {})
    techs[tech_key].setdefault("constraints", {})
    techs[tech_key]["constraints"]["resource"] = f"file={Path(csv_path).as_posix()}"  # => referencia a CSV
    # (opcional) si no definiste antes: techs[tech_key].setdefault("constraints", {}).setdefault("resource_unit","energy_per_cap")

    # => escribe de vuelta en payload
    payload["techs"] = techs
    locations[loc_name] = payload


In [10]:
data["locations"] = locations
YML_OUT.write_text(yaml.dump(data, sort_keys=False, allow_unicode=True), encoding="utf-8")
print(f"✅ YAML actualizado: {YML_OUT} (con resource -> pv_profiles_cf/<location>.csv)")

✅ YAML actualizado: locations_solares_with_resource.yml (con resource -> pv_profiles_cf/<location>.csv)


In [30]:
YML_IN = Path("../model_config/locations_solares.yml")
data = yaml.safe_load(YML_IN.read_text(encoding="utf-8"))
locations = data.get("locations", {})

for name, payload in locations.items():
    coords = payload.get("coordinates", None)
    lon, lat = float(coords[0]), float(coords[1])
    locations[name]['coordinates'] = {"lon": lon, "lat": lat}

data["locations"] = locations
yaml_text = yaml.dump(data, sort_keys=False, allow_unicode=True)

# Reemplazar el bloque de coordinates con formato inline
yaml_text = re.sub(
    r"coordinates:\s*\n\s*lon: ([^\n]+)\n\s*lat: ([^\n]+)",
    r"coordinates: {lon: \1,  lat: \2}",
    yaml_text
)

Path("locations_new.yml").write_text(yaml_text, encoding="utf-8")


20251

In [34]:
import pandas as pd
from pathlib import Path

carpeta = Path("../model_config/pv_profiles_cf")
print(carpeta)
for csv in carpeta.glob("*.csv"):
    print(csv)
    loc = csv.stem  # p.ej. PV_ALICAHUE
    df = pd.read_csv(csv)
    if "resource" in df.columns:
        df = df.rename(columns={"resource": loc})
        df.to_csv(csv, index=False)

../model_config/pv_profiles_cf
../model_config/pv_profiles_cf/PV_LOMA_LOS_COLORADOS.csv
../model_config/pv_profiles_cf/PV_SANTIAGO_SOLAR.csv
../model_config/pv_profiles_cf/PV_PALTO_SUNLIGHT.csv
../model_config/pv_profiles_cf/PV_FUSTER_DEL_VERANO.csv
../model_config/pv_profiles_cf/PV_COLINA.csv
../model_config/pv_profiles_cf/PV_VILLA_MOSCOSO.csv
../model_config/pv_profiles_cf/PV_PULLALLI.csv
../model_config/pv_profiles_cf/PV_TRANQUE_DEL_SOL.csv
../model_config/pv_profiles_cf/PV_OCOA.csv
../model_config/pv_profiles_cf/PV_PEDREROS_SOLAR.csv
../model_config/pv_profiles_cf/PV_RODEO.csv
../model_config/pv_profiles_cf/PV_LA_LIGUA.csv
../model_config/pv_profiles_cf/PV_EL_BOCO.csv
../model_config/pv_profiles_cf/PV_CATAN_SOLAR.csv
../model_config/pv_profiles_cf/PV_ENCON_SOLAR.csv
../model_config/pv_profiles_cf/PV_ANUNUCA_DEL_VERANO.csv
../model_config/pv_profiles_cf/PV_SANTA_INES_SOLAR.csv
../model_config/pv_profiles_cf/PV_MESETA_DE_LOS_ANDES.csv
../model_config/pv_profiles_cf/PV_PARQUE_SOLAR_OV

In [2]:
import pandas as pd
from pathlib import Path

carpeta = Path("../model_config/pv_profiles_cf")

for csv in carpeta.glob("*.csv"):
    df = pd.read_csv(csv)

    # Parsear a datetime (tz-aware en UTC) y luego quitar tz
    s = pd.to_datetime(df["time"], utc=True, errors="coerce").dt.tz_localize(None)
    if s.isna().any():
        raise ValueError(f"Timestamps inválidos en {csv}")

    # Ordenar y asegurar formato exacto
    df = df.assign(time=s).sort_values("time")
    df["time"] = df["time"].dt.strftime("%Y-%m-%d %H:%M:%S")

    # (Opcional) comprobar que todos terminan en :30 si tu resolución es 1H a HH:30
    if not (df["time"].str.slice(-5) == "00:30").all():
        # si necesitas desplazar para que queden a HH:30:
        df_time = pd.to_datetime(df["time"])
        df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")

    df.to_csv(csv, index=False)
    print(f"OK -> {csv.name}")

/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarnin

OK -> PV_LOMA_LOS_COLORADOS.csv
OK -> PV_SANTIAGO_SOLAR.csv
OK -> PV_PALTO_SUNLIGHT.csv
OK -> PV_FUSTER_DEL_VERANO.csv
OK -> PV_COLINA.csv
OK -> PV_VILLA_MOSCOSO.csv
OK -> PV_PULLALLI.csv
OK -> PV_TRANQUE_DEL_SOL.csv
OK -> PV_OCOA.csv
OK -> PV_PEDREROS_SOLAR.csv


/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarnin

OK -> PV_RODEO.csv
OK -> PV_LA_LIGUA.csv
OK -> PV_EL_BOCO.csv
OK -> PV_CATAN_SOLAR.csv
OK -> PV_ENCON_SOLAR.csv
OK -> PV_ANUNUCA_DEL_VERANO.csv
OK -> PV_SANTA_INES_SOLAR.csv
OK -> PV_MESETA_DE_LOS_ANDES.csv
OK -> PV_PARQUE_SOLAR_OVEJERIA.csv
OK -> PV_DONA_CARMEN_SOLAR.csv


/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarnin

...

/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarnin

OK -> PV_CONCON.csv
OK -> PV_EL_INGENIO.csv
OK -> PV_VALPARAISO.csv
OK -> PV_EL_HUASO.csv
OK -> PV_CASUTO.csv
OK -> PV_NORTE_CHICO.csv
OK -> PV_SANTA_TERESA.csv
OK -> PV_LONCURA.csv
OK -> PV_LOS_JOTES.csv
OK -> PV_LLIU_LLIU.csv


/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
/tmp/ipykernel_177205/3100885499.py:22: FutureWarnin

OK -> PV_SANTA_TERESITA.csv
OK -> PV_EL_LITRE_SOLAR.csv
OK -> PV_FILOMENA_SOLAR.csv
OK -> PV_SOLARPACK_VILLA_ALEMANA.csv
OK -> PV_ALTOS_DE_TIL_TIL.csv
OK -> PV_YAHUTELA.csv
OK -> PV_TUCUQUERE.csv
OK -> PV_CANONES_SUNLIGHT.csv
OK -> PV_PANQUEHUE_II.csv
OK -> PV_TORTOLAS_DEL_VERANO.csv
OK -> PV_LOS_PALTOS.csv


/tmp/ipykernel_177205/3100885499.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["time"] = (df_time.dt.floor("H") + pd.Timedelta(minutes=30)).dt.strftime("%Y-%m-%d %H:%M:%S")
